# HHL algorithm

The algorithm tries to solve the system of linear equations that can be represented as:

$$A\vec{x}=\vec{b}$$

The algorithm was proposed by [Aram W. Harrow, Avinatan Hassidim, and Seth Lloyd](https://arxiv.org/abs/0811.3171). It assumes that the eigenvalues ($\{\lambda_j,j=1,N\}$) and eigenstates ($\{u_j,j=1,N\}$) of matrix A of dimension $NxN$ are known. 

If $$\vec{b}=\sum_{j=1}^{M}\beta_j|u_j\rangle$$
and $C$ is a constant such as $C<\lambda_j$, then $\vec{x}$ can be approximated as

$$|x\rangle \approx C \sum_{j=1}^M \frac{\beta_j}{\lambda_j}|u_j\rangle$$

The algorithm uses three registers:

1. S An ancilla qubit to get the solution
2. C or clock register
3. b to store $\vec{b}$

So, the init state is $|0\rangle_S|0\rangle_C|0\rangle_b$


The steps of algorithm are:

1. Prepare an initial state $|b\rangle_{b}$ 
2. Make a phase estimation of A
3. Make control rotations on S
4. Uncompute 2
5. Measure S. if result is $|1>$ then return the register I, else goto step 1.
6. Calculate the expectation value of one operator on b which is equivalent to $\langle x|Operator|x\rangle$


This exercise implements a simpler version of the HHL algorithm from [Coles et al.](https://arxiv.org/abs/1804.03719). The system of linear equations to solve is

$\frac{1}{2}\begin{bmatrix}3&1\\1&3\end{bmatrix}\vec{x}=\begin{bmatrix}1\\0\end{bmatrix}$

The eigenvalues of this matrix are $\lambda_1=1$ and $\lambda_2=2$, with eigenvectors $|-\rangle=\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$ and $|+\rangle=\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$ 


In [1]:
from projectq.cengines import MainEngine

In [2]:
from projectq.ops import Ry,Measure,All,H,X,Swap,CNOT
from projectq.meta import Compute,Uncompute,Control

In [3]:
from projectq.ops import QubitOperator

Select *case* as:

0. $b=|0\rangle$
1. $b=|+\rangle=\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle$
2. $b=|-\rangle=\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle$

In [4]:
case=2 #select case=0, |0>; case =1, |+>; case=2, |->

This is a helper function to print the final state

In [5]:
def Display(string):
    from IPython.display import display, Markdown
    display(Markdown(string))

In [6]:
def get_state_as_str(eng,qubits,cheat=False):
    import numpy as np
    s="$"
    if (cheat):
        print("Cheat: ", eng.backend.cheat())
    if (len(qubits)==1):
        for i in range(2):
            #print("bits:%d%s"%(i,bits))
            a=eng.backend.get_amplitude("%d"%(i),qubits)
            if (a.real!=0)|(a.imag!=0):
                if s!="$":
                    s=s+"+"
                a="({:.2f})".format(a)
                s=s+"%s|%d>"%(a,i)

    else:
        for j in range(2**(len(qubits)-1)):
            bits=np.binary_repr(j,width=len(qubits)-1)
            #print("Bits:",j,bits)
            for i in range(2):
                #print("bits:%d%s"%(i,bits))
                a=eng.backend.get_amplitude("%d%s"%(i,bits[-1::-1]),qubits)
                if (a.real!=0)|(a.imag!=0):
                    if s!="$":
                        s=s+"+"
                    a="({:.2f})".format(a)
                    s=s+"%s|%s>_S|%s>_C|%d>_b"%(a,bits[0],bits[1:],i)
                #print(s)
    s=s+"$"
    #Display(s)
    return(s)

For this version of the algorithm, the Quantum Phase Estimation can be simplified to a very few operations.

In [7]:
def QPE(b,C):
        H | b
        CNOT | (b,C[0])
        CNOT | (C[0],C[1])
        X | C[0]
        Swap | (C[0],C[1])



This is the function to apply the controlled rotations on qubit *S*

In [8]:
def ControledRotation(eng,C,S):
    import math
    Theta_1=math.pi
    Theta_2=math.pi/3
    
    with Control(eng,C[0]):
        Ry(Theta_1) | S
    with Control(eng,C[1]):
        Ry(Theta_2) |S


In [9]:
def initialize(eng,qureg,case):
    b=qureg[0]
    if (case == 2):
        X | b
    if (case >0):
        H | b
    eng.flush()
    Display("Solving for vector:%s"%(get_state_as_str(eng,qureg)))


This is the main loop. It has to be executed until the result of the measurement of quantum register **S** is $|1\rangle$. The main loop has three steps:

1. Initialize quantum register $|b\rangle$
2. Compute QPE
3. Computer Controlled rotations
4. Uncompute QPE

In [10]:
result=0
while (result==0):
    eng=MainEngine()
    b=eng.allocate_qureg(1)
    C=eng.allocate_qureg(2)
    S=eng.allocate_qureg(1)
    qureg=b+C+S
    
    initialize(eng,qureg,case)
    
    with Compute(eng):
        QPE(b,C)
    
    ControledRotation(eng,C,S)
    
    Uncompute(eng)

    Measure | S
    eng.flush()
    Display("Ancilla measurement: %d"%int(S))
    result=int(S)
    if result==0:
        All(Measure) | C
        Measure |b
        eng.flush()
        del eng

Solving for vector:$(0.71+0.00j)|0>_S|00>_C|0>_b+(-0.71+0.00j)|0>_S|00>_C|1>_b$

Ancilla measurement: 1

This helper funtion, solves the system and calculates the expectation values for the Pauli's Matrices. 

In [11]:
def solve(case=1):
    import numpy as np
    a = np.array([[1.5,0.5], [0.5,1.5]])
    if (case==0):
        b = np.array([1,0])
    if (case==1):
        b = np.array([1,1]/np.sqrt(2))
    if (case==2):
        b = np.array([1,-1]/np.sqrt(2))    
    x = np.linalg.solve(a, b)
    sigmax=np.array([[0,1],[1,0]])
    sigmay=np.array([[0,-1j],[1j,0]])
    sigmaz=np.array([[1,0],[0,-1]])
    norm=np.linalg.norm(x)
    Esx=np.dot(x,np.dot(sigmax,(x.reshape((2,1)))))[0]/norm**2
    Esy=np.dot(x,np.dot(sigmay,(x.reshape((2,1)))))[0]/norm**2
    Esz=np.dot(x,np.dot(sigmaz,(x.reshape((2,1)))))[0]/norm**2
    return Esx,Esy,Esz,x

OK. If the program has reached this point is because the measurement of quantum register **s** is 1. Now, using ProjectQ capabilities, it is possible to calculate the final state vector and the expected valued of Pauli operators.

Remember that the expectation value of an operator M for an state $|x\rangle$ (which is normalised) is defined by:

$$<M>=<x|M|x>$$

In [12]:
Display("X_quantum:%s"%(get_state_as_str(eng,b+C+S)))
Esx,Esy,Esz,X=solve(case)
Display("X_classical: %s"%[ "{:0.2f}".format(x) for x in X ])
QEsx=eng.backend.get_expectation_value(QubitOperator("X0"),b)
QEsy=eng.backend.get_expectation_value(QubitOperator("Y0"),b)
QEsz=eng.backend.get_expectation_value(QubitOperator("Z0"),b)
Display("Calculated $< \sigma_{x}>$=%.3f. Must be %.3f"%(QEsx,Esx.real))
Display("Calculated $< \sigma_{y}>$=%.3f. Must be %.3f"%(QEsy,Esy.real))
Display("Calculated $< \sigma_{z}>$=%.3f. Must be %.3f"%(QEsz,Esz.real))
    

X_quantum:$(0.71+0.00j)|1>_S|00>_C|0>_b+(-0.71+0.00j)|1>_S|00>_C|1>_b$

X_classical: ['0.71', '-0.71']

Calculated $< \sigma_{x}>$=-1.000. Must be -1.000

Calculated $< \sigma_{y}>$=0.000. Must be 0.000

Calculated $< \sigma_{z}>$=0.000. Must be 0.000

Delete everything before end

In [13]:
All(Measure) | C
Measure |b
eng.flush()
del eng        
